## Experiment description

This run is done using CFD. Used an 80% train test split. 
The CFD data has 56 samples where each sample has 27 features. There are 10 inputs and 17 outputs. All inputs and outputs were used. Tested an MLP and a 2 layer DL architecture. The loss does not look great but the 12 predictions in the test set seem not too bad. 


Train_X.........(44, 10)

Train_y.........(44, 17)

Test_X.........(12, 10)

Test_y.........(12, 17)

Results are at the bottom. 

In [1]:
################################################
##
## regression with DL for system control
##
################################################

import torch
import numpy as np
import pandas as pd
import sklearn
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

#######################################################

N_EPOCHS = 10000
batch_size = 5
learning_rate = 0.001 ## 1e-5 

#######################################################

np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

#######################################################


def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
    print('Training loss:', loss_fn(model(X_train_tr), y_train_tr))
    
#######################################################



In [2]:

CFD_raw_data = pd.read_csv('CFD.6.2022.csv') 


In [3]:
headers_list = CFD_raw_data.columns.values.tolist()
print(headers_list)
print(len(headers_list))
print(headers_list[27])

for i, name in enumerate(headers_list):
    print((i, name))

['index', 'i_pul_coal_inj_kg_thm', 'i_nat_gas_inj_kg_thm', 'i_nat_gas_t_k', 'i_o2_vol_perce', 'i_bf_windrate_nm3_hr', 'i_hb_moist_g_nm3', 'i_hot_blast_press_pa', 'i_hot_blast_temp_k', 'i_coke_weight_kg', 'i_ore_weight_kg', 'o_tuyere_exit_velo_m_s', 'o_tuyere_t_k', 'o_raceway_flame_temp_k', 'o_raceway_coal_burn_perce', 'o_raceway_volume_m', 'o_raceway_depth _m', 'o_shaft_co_utiliz', 'o_shaft_h2_utiliz', 'o_shaft_top_gas_temp_c', 'o_shaft_press_drop_pa', 'o_shaft_coke_rate_kg_thm', 'o_shaft_cohesive_zone_tip_height_m', 'o_shaft_cohes_zone_root_height_m', 'o_shaft_co_v_perc', 'o_shaft_co2_v_perc', 'o_shaft_h2_v_perce', 'o_shaft_n2_v_perc']
28
o_shaft_n2_v_perc
(0, 'index')
(1, 'i_pul_coal_inj_kg_thm')
(2, 'i_nat_gas_inj_kg_thm')
(3, 'i_nat_gas_t_k')
(4, 'i_o2_vol_perce')
(5, 'i_bf_windrate_nm3_hr')
(6, 'i_hb_moist_g_nm3')
(7, 'i_hot_blast_press_pa')
(8, 'i_hot_blast_temp_k')
(9, 'i_coke_weight_kg')
(10, 'i_ore_weight_kg')
(11, 'o_tuyere_exit_velo_m_s')
(12, 'o_tuyere_t_k')
(13, 'o_raceway

In [4]:
CFDdata_np = CFD_raw_data.to_numpy()
print(CFDdata_np)
print(CFDdata_np.shape)

[[ 1.          0.         37.9        ...  0.1941631   0.0438327
   0.5448123 ]
 [ 2.          0.         37.9        ...  0.2186134   0.04034286
   0.5149459 ]
 [ 3.          0.         37.9        ...  0.2353341   0.03772617
   0.4849808 ]
 ...
 [54.         70.          0.         ...  0.2279862   0.02182447
   0.5193577 ]
 [55.         70.          0.         ...  0.2387356   0.02094042
   0.4884186 ]
 [56.         70.          0.         ...  0.2464429   0.01986916
   0.4608655 ]]
(56, 28)


In [5]:

#######################################################

input_indeces  = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
output_indeces = [11, 12 ,13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

#######################################################

X = CFDdata_np[:, input_indeces]

y = CFDdata_np[:, output_indeces]

print(X.shape)
print(y.shape)

#######################################################

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#######################################################

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(56, 10)
(56, 17)
(44, 10)
(12, 10)
(44, 17)
(12, 17)


In [6]:


## fix data type
X_train  = X_train.astype(np.float32)
X_test   = X_test.astype(np.float32)
y_train  = y_train.astype(np.float32)
y_test   = y_test.astype(np.float32)

X_train_tr  = torch.from_numpy(X_train)
X_test_tr   = torch.from_numpy(X_test)
y_train_tr  = torch.from_numpy(y_train)
y_test_tr   = torch.from_numpy(y_test)

In [7]:
#######################################################
## define dataset

train_ds = TensorDataset(X_train_tr, y_train_tr)

#######################################################
## define dataloader

train_dl = DataLoader(train_ds, batch_size, shuffle=True)


#######################################################

In [8]:
#######################################################

class SimpleNet(nn.Module):
    ## initialize the layers
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(10, 12)
        self.act1    = nn.Tanh() 
        self.linear2 = nn.Linear(12, 17)
    
    ## perform inference
    def forward(self, x):
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        return x
    
#############################################################


class DL_Net(nn.Module):
    ## initialize the layers
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(10, 32)
        self.act1    = nn.Tanh() 
        self.linear2 = nn.Linear(32, 24)
        self.act2    = nn.Tanh() 
        self.linear3 = nn.Linear(24, 17)
    
    ## perform inference
    def forward(self, x):
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        x = self.act2(x)
        x = self.linear3(x)
        return x
    
#############################################################

## model = SimpleNet()
model = DL_Net()
## opt = torch.optim.SGD(   model.parameters(), lr=learning_rate  )
opt = torch.optim.Adam(   model.parameters(), lr=learning_rate   )
loss_fn = F.mse_loss

In [9]:
fit(N_EPOCHS, model, loss_fn, opt)



Training loss: tensor(848382976., grad_fn=<MseLossBackward0>)


In [10]:
for i in range(len(X_test_tr)):
    print("**************************************************")
    print("preds, real")
    preds = model(X_test_tr[i])
    np_pred = preds.detach().numpy()
    np_real = y_test_tr[i].detach().numpy()
    for j in range(len(np_pred)):
        print((np_pred[j], np_real[j]))
    

## preds = model(X_test_tr)
## print(preds.detach().numpy())
## print(y_test_tr.detach().numpy())

**************************************************
preds, real
(136.78838, 133.4865)
(1413.6077, 1440.96)
(2131.0513, 2202.8)
(33.182484, 0.0)
(0.21339579, 0.1965359)
(0.8265417, 0.8111)
(47.171326, 47.20085)
(42.937687, 42.99392)
(114.27447, 118.8785)
(2251.105, 108434.33)
(433.76126, 462.73654)
(31.160686, 31.83963)
(9.91544, 10.649022)
(0.22316808, 0.2171919)
(0.20103799, 0.1941631)
(0.062504366, 0.043832704)
(0.49172464, 0.5448123)
**************************************************
preds, real
(136.78838, 133.9054)
(1413.6077, 1444.46)
(2131.0513, 2214.316)
(33.182484, 0.0)
(0.21339579, 0.1965359)
(0.8265417, 0.8111)
(47.171326, 48.77197)
(42.937687, 43.93872)
(114.27447, 104.5031)
(2251.105, 101647.98)
(433.76126, 454.2938)
(31.160686, 31.75963)
(9.91544, 9.713824)
(0.22316808, 0.2119294)
(0.20103799, 0.2017688)
(0.062504366, 0.043137867)
(0.49172464, 0.543164)
**************************************************
preds, real
(136.78838, 157.34734)
(1413.6077, 1367.4567)
(2131.0513, 

In [11]:
print('Test loss:', loss_fn(model(X_test_tr), y_test_tr))

Test loss: tensor(769857664., grad_fn=<MseLossBackward0>)
